In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import os
import pickle

localpath_prefix='/Users/haizhao/OneDrive - Columbia University Irving Medical Center'


In [2]:
%%time
#@title Load Human_ZEPPI data files; takes ~10s after files are ready to read

fileread1=localpath_prefix + '/WORK/PrePPI_MI/Human_af/human_af_lrg28_ZIPPI_06132023.csv'
fileread2=localpath_prefix + '/WORK/PrePPI_MI/Human_af/human_af_lrg4l28_ZEPPI_08222023.csv' 
fileread3=localpath_prefix + '/WORK/PrePPI_MI/Human_af/human_af.lrg1l4_ZEPPI_03142024.csv'
fileread4=localpath_prefix + '/WORK/PrePPI_MI/Human_af/human_af.lrl1_ZEPPI_03142024.csv'

df_g28 = pd.read_csv(fileread1,sep=',',skiprows=0,usecols=['Uni_pair','LR','Neff','Nifr','Zmaxmt'])
df_g4l28 = pd.read_csv(fileread2,sep=',',skiprows=0,usecols=['Uni_pair','LR','Neff','Nifr','Zmaxmt'])
df_g1l4 = pd.read_csv(fileread3,sep=',',skiprows=0,usecols=['Uni_pair','LR','Neff','Nifr','Zmaxmt'])
df_l1 = pd.read_csv(fileread4,sep=',',skiprows=0,usecols=['Uni_pair','LR','Neff','Nifr','Zmaxmt'])

df_lr_g0 = pd.concat([df_g28, df_g4l28, df_g1l4, df_l1], ignore_index=True)

print("Number of total_PPIs from ZEPPI:",len(df_lr_g0))


Number of total_PPIs from ZEPPI: 9863206
CPU times: user 4.09 s, sys: 582 ms, total: 4.67 s
Wall time: 5.03 s


In [7]:
# Read DS predictions; takes <1min after files are ready. 

filetoread=localpath_prefix+'/WORK/PrePPI_MI/DS/human/human_part1_withhomo.tsv'
df_DS=pd.read_csv(filetoread,sep='\t',names=["Uniprot1", "Uniprot2",'DS'])
df_DS = df_DS.dropna()
df_DS['Uni_pair']=df_DS['Uniprot1']+"_"+ df_DS['Uniprot2']
len(df_DS)

59550146

In [8]:
#--- read human HINT HQ-LC and allDBs; 

def readcsv_return_pair(filename,sep_symbol,skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN) #names=["hfpd_ID", "Uni"]
    ref_pairs=df0['UniprotID_A_UniprotID_B'].tolist()
    return ref_pairs

def readcsv2_return_sortedpair(filename,sep_symbol,skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN) #names=["hfpd_ID", "Uni"]
    ref_pairs=tuple(zip(df0['UniprotID_A'], df0['UniprotID_B']))
    ref_pairs_sorted=[]
    for i in ref_pairs:
        i = '_'.join(sorted(i))
        if i not in ref_pairs_sorted:
            ref_pairs_sorted.append(i)
    return ref_pairs_sorted

HINT_hq_bi_LC_pairs = readcsv2_return_sortedpair(localpath_prefix+'/WORK/Human_DB/Expts/2021HINT_Human_lcb_hq.txt','\t')
df_hqLC = pd.DataFrame(HINT_hq_bi_LC_pairs, columns=['Uni_pair'])
print("Number of HINT_LC_humanPPI:",len(df_hqLC))

#--- Read allExpDBs database
allExpPhy_file = localpath_prefix+'/WORK/Human_DB/human_ExpPhy_details.csv'
allExpPhy_df = pd.read_csv(allExpPhy_file,skiprows=0,usecols=[0],names=["Uni_pair"])
print("Number of allExpDB_PPIs:",len(allExpPhy_df))

allExpDB_file = localpath_prefix+'/WORK/Human_DB/human_allExpDB.csv'
allExpDB_df = pd.read_csv(allExpDB_file,skiprows=0,usecols=[0],names=["Uni_pair"])
print("Number of allExpPhy_PPIs:",len(allExpDB_df))

#--- Read STRING database
def readcsv2_return_pair(filename,sep_symbol,skipN=0):
    df0=pd.read_csv(filename,sep=sep_symbol,skiprows=skipN)
    return df0['UniprotID_A_UniprotID_B'].tolist()

toreadfile1 = localpath_prefix+'/WORK/Human_DB/Expts/2022String_9606.physical.links.experimental.sortedpair.nonred.csv'

STRING_Human_exp_pairs = readcsv2_return_pair(toreadfile1,',')
df_STRING = pd.DataFrame(STRING_Human_exp_pairs, columns=['Uni_pair'])
print("Number of STRING-Physical Human (exp >0): ",len(STRING_Human_exp_pairs))


Number of HINT_LC_humanPPI: 16167
Number of allExpDB_PPIs: 766045
Number of allExpPhy_PPIs: 1200001
Number of STRING-Physical Human (exp >0):  272379


In [9]:
#@title Define function to assign labels; 

def AssignLabels(df,pos,allExpDB):
    df_inPos = df[df['Uni_pair'].isin(pos)].copy()
    print("Number of input_PPIs in Pos_set:",len(df_inPos))

    ratio=1000; n=ratio * len(df_inPos)
    df_notinExpDB = df[~df['Uni_pair'].isin(allExpDB)].copy()
    if n < len(df_notinExpDB):
        df_notinExpDB_sampled = df_notinExpDB.sample(n)
    else:
        df_notinExpDB_sampled = df_notinExpDB

    print("Number of input_PPIs in ExpDBs:",len(df)-len(df_notinExpDB))
    print("Number of input_PPIs not in ExpDBs:",len(df_notinExpDB))

    df_inPos['label'] = 1
    df_notinExpDB_sampled['label'] = 0
    df_inPosNeg = pd.concat([df_inPos,df_notinExpDB_sampled])
    return df_inPosNeg



In [10]:
#@title Assign labels; Prepare features for ZEPPI 

df_ZEPPI_inHINT_Unknown_totrain = AssignLabels(df_lr_g0,df_hqLC['Uni_pair'],allExpDB_df['Uni_pair'])
df_ZEPPI_inHINT_Unknown_totrain = df_ZEPPI_inHINT_Unknown_totrain.reset_index(drop=True)


Number of input_PPIs in Pos_set: 7582
Number of input_PPIs in ExpDBs: 108287
Number of input_PPIs not in ExpDBs: 9754919


In [ ]:
#@title Assign labels; Prepare features for DS (takes >10min due to large data size)

df_DS_inSTRING_Unknown_totrain = AssignLabels(df_DS,df_STRING['Uni_pair'],allExpDB_df['Uni_pair'])
df_DS_inSTRING_Unknown_totrain = df_DS_inSTRING_Unknown_totrain.reset_index(drop=True)


Number of input_PPIs in Pos_set: 45465
Number of input_PPIs in ExpDBs: 228852
Number of input_PPIs not in ExpDBs: 59321294


KeyboardInterrupt: 

In [ ]:
#@title Check training data of ZEPPI and DS(TT) before starting training 

for i in [df_ZEPPI_inHINT_Unknown_totrain, df_DS_inSTRING_Unknown_totrain]:
    data = i 

    # Check NaN values
    nan_values = data.isna().any()

    # Print columns with NaN values
    rows_with_nan = data[data.isna().any(axis=1)]

    print("Rows with NaN values:")
    print(rows_with_nan)


Rows with NaN values:
Empty DataFrame
Columns: [Uni_pair, Neff, Nifr, LR, Zmaxmt, label]
Index: []


In [7]:
#@title Import Discretized Naive Bayes (Naive, Simple and Independent)

import sys, os
sys.path.append(os.path.abspath(".."))

import importlib
from src import Bayes
importlib.reload(Bayes)

from src.Bayes import DiscretizedNaiveBayes_IndependentFeatures
from src.Bayes import DiscretizedNaiveBayes_JointBins


In [13]:
%%time
#@title Split training and testing set; Train ZEPPI_LR

from sklearn.model_selection import train_test_split

#X_train_, X_test_, y_train_, y_test_ = train_test_split(data[['Zmaxmt']], data['label'], test_size=0.001, random_state=40)
X_train_, X_test_, y_train_, y_test_ = train_test_split(data[['Zmaxmt','Neff','Nifr']], data['label'], test_size=0.001, random_state=40)

#X_trainB3, X_testB3, y_trainB3, y_testB3 = train_test_split(data[['COV', 'SIZ',  'OL', 'OS']], data['label'], test_size=0.2, random_state=40)

train_indices = np.sort(X_train_.index)
train_UniPairs = data.loc[train_indices, "Uni_pair"]
test_indices = np.sort(X_test_.index)

X_test_sorted = X_test_.sort_index()
y_test_sorted = y_test_.sort_index()

""""""
X_train0 = X_train_.to_numpy()
y_train0 = y_train_.to_numpy()
X_test0 = X_test_.to_numpy()
y_test0 = y_test_.to_numpy()
X_test_sorted = X_test_sorted.to_numpy()
y_test_sorted = y_test_sorted.to_numpy()

#nbn_ZP = DiscretizedNaiveBayes_IndependentFeatures()
nbn_ZP =DiscretizedNaiveBayes_JointBins()

nbn_ZP.fit(X_train0, y_train0)

#data_test = data.loc[~data.index.isin(train_indices)]
#print(len(X_test_sorted),len(data_test),data_test.index)

#data_test['LR_ZP'] = nbn1.predict(X_test_sorted)
#data_test['LRSM_LRZP'] = data_test['LR'] * data_test['LR_ZP']


CPU times: user 9.56 s, sys: 1.49 s, total: 11.1 s
Wall time: 15.6 s


In [ ]:
# Save the trained ZEPPI_LR model using pickle
import pickle
with open('../data/human_ZP_BayesB3_ratio1000.pkl', 'wb') as f:
    pickle.dump(nbn_ZP, f)


In [ ]:
%%time
#@title Split training and testing set; Train DS(TT)_LR
data2 = df_DS_inSTRING_Unknown_totrain

from sklearn.model_selection import train_test_split

X_train_, X_test_, y_train_, y_test_ = train_test_split(data2[['DS']], data2['label'], test_size=0.001, random_state=40)

train_indices = np.sort(X_train_.index)
train_UniPairs = data2.loc[train_indices, "Uni_pair"]
test_indices = np.sort(X_test_.index)

X_test_sorted = X_test_.sort_index()
y_test_sorted = y_test_.sort_index()

""""""
X_train0 = X_train_.to_numpy()
y_train0 = y_train_.to_numpy()
X_test0 = X_test_.to_numpy()
y_test0 = y_test_.to_numpy()
X_test_sorted = X_test_sorted.to_numpy()
y_test_sorted = y_test_sorted.to_numpy()

nbn2 = DiscretizedNaiveBayes_IndependentFeatures()
nbn2.fit(X_train0, y_train0)


unique_classes [0 1]
dict_keys([36, 19, 0, 3, 1, 20, 8, 9, 33, 5, 17, 38, 25, 23, 2, 11, 27, 6, 29, 13, 7, 22, 16, 37, 4, 31, 34, 18, 10, 24, 26, 12, 15, 35, 21, 14, 30, 28, 32]) dict_values([179533, 9827960, 24831769, 461698, 2269159, 74071, 147846, 219575, 405375, 2202831, 83992, 57758, 60335, 57533, 2037017, 105921, 61571, 216151, 108861, 93985, 190241, 58557, 79201, 99579, 277285, 76086, 70899, 168371, 119964, 58149, 67830, 117714, 81197, 86228, 62406, 85224, 67769, 64776, 85109])
dict_keys([1, 0, 9, 35, 2, 33, 36, 37, 19, 23, 29, 3, 4, 5, 31, 12, 30, 11, 32, 10, 13, 28, 25, 24, 22, 34, 26, 16, 27, 17, 18, 6, 7, 20, 38, 14, 15, 8, 21]) dict_values([2897, 18698, 459, 1096, 1942, 925, 1642, 1792, 2320, 263, 468, 1053, 748, 1252, 483, 333, 437, 340, 564, 367, 315, 360, 288, 312, 259, 831, 298, 283, 347, 274, 276, 525, 537, 279, 922, 287, 272, 413, 271])
CPU times: user 22.7 s, sys: 3min 15s, total: 3min 37s
Wall time: 5min 15s


In [ ]:
# Save the trained DS(TT)_LR model using pickle

with open('../data/human_DSwthomo_Bayes_STRING.pkl', 'wb') as f:
    pickle.dump(nbn2, f)

In [9]:
from src.evaluation import ClassificationEvaluator


In [ ]:
# Evaluate the performance

data2_test = data2.loc[~data2.index.isin(train_indices)]
data2_test['LR_DS'] = nbn2.predict(X_test_sorted)

evaluator2 = ClassificationEvaluator('$LR^{DS}$ on HINT-LC', data2_test['label'].to_numpy(), data2_test['LR_DS'].to_numpy())
evaluator2.plot_roc_curve_ratio()
evaluator2.plot_prc_curve_ratio()
